**Bagian 1**

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving customer_data.csv to customer_data.csv


In [ ]:
import pandas as pd

df = pd.read_csv('customer_data.csv')

df.head()


,ID,Year_Birth,Education,Income,Dt_Customer,Recency,NumWebPurchases,NumStorePurchases,NumWebVisitsMonth
0,5524,1957,Graduation,58138.0,9/4/2012,58,8,4,7
1,2174,1954,Graduation,46344.0,3/8/2014,38,1,2,5
2,4141,1965,Graduation,71613.0,8/21/2013,26,8,10,4
3,6182,1984,Graduation,26646.0,2/10/2014,26,2,4,6
4,5324,1981,PhD,58293.0,1/19/2014,94,5,6,5


In [ ]:
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values)



Missing Values:
 ID                   0
Year_Birth           0
Education            0
Income               0
Dt_Customer          0
Recency              0
NumWebPurchases      0
NumStorePurchases    0
NumWebVisitsMonth    0
dtype: int64


In [ ]:
df['Income'].fillna(df['Income'].median(), inplace=True)
df['Education'].fillna(df['Education'].mode()[0], inplace=True)

<ipython-input-63-70328c431e78>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Income'].fillna(df['Income'].median(), inplace=True)
<ipython-input-63-70328c431e78>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

In [ ]:
Q1 = df['Income'].quantile(0.25)
Q3 = df['Income'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['Income'] < lower_bound) | (df['Income'] > upper_bound)]
print("Outliers:\n", outliers)

df = df[(df['Income'] >= lower_bound) & (df['Income'] <= upper_bound)]


Outliers:
 Empty DataFrame
Columns: [ID, Year_Birth, Education, Income, Dt_Customer, Recency, NumWebPurchases, NumStorePurchases, NumWebVisitsMonth]
Index: []


In [ ]:
print(df['Education'].unique())

df['Education'] = df['Education'].replace({'graduated': 'Graduated'})


['Graduation' 'PhD' 'Master' 'Basic' '2n Cycle']


In [ ]:
duplicates = df[df.duplicated(subset='ID')]
print("Duplicate Records:\n", duplicates)

df = df.drop_duplicates(subset='ID')


Duplicate Records:
 Empty DataFrame
Columns: [ID, Year_Birth, Education, Income, Dt_Customer, Recency, NumWebPurchases, NumStorePurchases, NumWebVisitsMonth]
Index: []


In [ ]:
print(df.info())
print(df.describe())


<class 'pandas.core.frame.DataFrame'>
Index: 2232 entries, 0 to 2239
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 2232 non-null   int64  
 1   Year_Birth         2232 non-null   int64  
 2   Education          2232 non-null   object 
 3   Income             2232 non-null   float64
 4   Dt_Customer        2232 non-null   object 
 5   Recency            2232 non-null   int64  
 6   NumWebPurchases    2232 non-null   int64  
 7   NumStorePurchases  2232 non-null   int64  
 8   NumWebVisitsMonth  2232 non-null   int64  
dtypes: float64(1), int64(6), object(2)
memory usage: 174.4+ KB
None
                 ID   Year_Birth         Income      Recency  NumWebPurchases  \
count   2232.000000  2232.000000    2232.000000  2232.000000      2232.000000   
mean    5590.736559  1968.792563   51630.926971    49.111111         4.097670   
std     3246.390817    11.990340   20601.679542    28.95289

In [ ]:
df.to_csv('customer_data_cleaned.csv', index=False)


In [ ]:
df_cleaned = pd.read_csv('customer_data_cleaned.csv')

print(df_cleaned.head())


     ID  Year_Birth   Education   Income Dt_Customer  Recency  \
0  5524        1957  Graduation  58138.0    9/4/2012       58   
1  2174        1954  Graduation  46344.0    3/8/2014       38   
2  4141        1965  Graduation  71613.0   8/21/2013       26   
3  6182        1984  Graduation  26646.0   2/10/2014       26   
4  5324        1981         PhD  58293.0   1/19/2014       94   

   NumWebPurchases  NumStorePurchases  NumWebVisitsMonth  
0                8                  4                  7  
1                1                  2                  5  
2                8                 10                  4  
3                2                  4                  6  
4                5                  6                  5  


**NO 1 ( A )**

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
from datetime import datetime

df_cleaned = pd.read_csv('customer_data_cleaned.csv')

df_cleaned['Dt_Customer'] = pd.to_datetime(df_cleaned['Dt_Customer'], format='%m/%d/%Y')

current_date = datetime.now()
df_cleaned['Years_Since_Registration'] = (current_date - df_cleaned['Dt_Customer']).dt.days / 365

df_cleaned['Age'] = current_date.year - df_cleaned['Year_Birth']

print(df_cleaned[['ID', 'Year_Birth', 'Age', 'Dt_Customer', 'Years_Since_Registration']].head())


     ID  Year_Birth  Age Dt_Customer  Years_Since_Registration
0  5524        1957   67  2012-09-04                 12.191781
1  2174        1954   70  2014-03-08                 10.684932
2  4141        1965   59  2013-08-21                 11.230137
3  6182        1984   40  2014-02-10                 10.756164
4  5324        1981   43  2014-01-19                 10.816438


**NO 1 ( B )**

In [ ]:
features = df_cleaned[['Income', 'Recency', 'NumWebPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'Years_Since_Registration', 'Age']]

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

kmeans = KMeans(n_clusters=3, random_state=42)
df_cleaned['Cluster'] = kmeans.fit_predict(features_scaled)

print(df_cleaned[['ID', 'Cluster', 'Income', 'Recency', 'NumWebPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'Years_Since_Registration', 'Age']].head())



     ID  Cluster   Income  Recency  NumWebPurchases  NumStorePurchases  \
0  5524        2  58138.0       58                8                  4   
1  2174        1  46344.0       38                1                  2   
2  4141        0  71613.0       26                8                 10   
3  6182        1  26646.0       26                2                  4   
4  5324        0  58293.0       94                5                  6   

   NumWebVisitsMonth  Years_Since_Registration  Age  
0                  7                 12.191781   67  
1                  5                 10.684932   70  
2                  4                 11.230137   59  
3                  6                 10.756164   40  
4                  5                 10.816438   43  


In [ ]:
numerical_cols = df_cleaned.select_dtypes(include=[np.number]).columns.tolist()

cluster_characteristics = df_cleaned.groupby('Cluster')[numerical_cols].mean()
print(cluster_characteristics)


                  ID   Year_Birth        Income    Recency  NumWebPurchases  \
Cluster                                                                       
0        5725.217125  1967.000000  72605.376911  49.584098         4.570336   
1        5590.722051  1972.099487  33410.512308  48.883077         2.014359   
2        5444.905473  1965.389718  58343.388060  48.966833         6.953566   

         NumStorePurchases  NumWebVisitsMonth  Years_Since_Registration  \
Cluster                                                                   
0                 8.434251           2.571865                 11.119702   
1                 3.092308           6.591795                 11.255326   
2                 7.351575           6.286899                 11.730734   

               Age  Cluster  
Cluster                      
0        57.000000      0.0  
1        51.900513      1.0  
2        58.610282      2.0  


**Bagian 2**

In [ ]:
import pandas as pd

df = pd.read_csv('laptop_data.csv')

df.head()


,Company,TypeName,Ram,Weight,Price,TouchScreen,Ips,Ppi,Cpu_brand,HDD,SSD,Gpu_brand,Os
0,Apple,Ultrabook,8,1.37,11.175755,0,1,226.983005,Intel Core i5,0,128,Intel,Mac
1,Apple,Ultrabook,8,1.34,10.776777,0,0,127.677940,Intel Core i5,0,0,Intel,Mac
2,HP,Notebook,8,1.86,10.329931,0,0,141.211998,Intel Core i5,0,256,Intel,Others
3,Apple,Ultrabook,16,1.83,11.814476,0,1,220.534624,Intel Core i7,0,512,AMD,Mac
4,Apple,Ultrabook,8,1.37,11.473101,0,1,226.983005,Intel Core i5,0,256,Intel,Mac


In [ ]:
df.isnull().sum()


,0
Company,0
TypeName,0
Ram,0
Weight,0
Price,0
TouchScreen,0
Ips,0
Ppi,0
Cpu_brand,0
HDD,0


In [ ]:
df['Ram'].fillna(df['Ram'].median(), inplace=True)
df['Weight'].fillna(df['Weight'].median(), inplace=True)
df['Price'].fillna(df['Price'].median(), inplace=True)

df['Company'].fillna(df['Company'].mode()[0], inplace=True)


<ipython-input-92-373b9f07666e>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Ram'].fillna(df['Ram'].median(), inplace=True)
<ipython-input-92-373b9f07666e>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [ ]:
df.drop_duplicates(inplace=True)


In [ ]:
Q1 = df['Price'].quantile(0.25)
Q3 = df['Price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_filtered = df[(df['Price'] >= lower_bound) & (df['Price'] <= upper_bound)]


In [ ]:
df['Company'] = df['Company'].replace('Acerr', 'Acer')


In [ ]:
df.to_csv('laptop_data_cleaned.csv', index=False)


In [ ]:
df_cleaned = pd.read_csv('laptop_data_cleaned.csv')

print(df_cleaned.head())


  Company   TypeName  Ram  Weight      Price  TouchScreen  Ips         Ppi  \
0   Apple  Ultrabook    8    1.37  11.175755            0    1  226.983005   
1   Apple  Ultrabook    8    1.34  10.776777            0    0  127.677940   
2      HP   Notebook    8    1.86  10.329931            0    0  141.211998   
3   Apple  Ultrabook   16    1.83  11.814476            0    1  220.534624   
4   Apple  Ultrabook    8    1.37  11.473101            0    1  226.983005   

       Cpu_brand  HDD  SSD Gpu_brand      Os  
0  Intel Core i5    0  128     Intel     Mac  
1  Intel Core i5    0    0     Intel     Mac  
2  Intel Core i5    0  256     Intel  Others  
3  Intel Core i7    0  512       AMD     Mac  
4  Intel Core i5    0  256     Intel     Mac  


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['TouchScreen'] = label_encoder.fit_transform(df['TouchScreen'])
df['Ips'] = label_encoder.fit_transform(df['Ips'])

df_encoded = pd.get_dummies(df, columns=['Company', 'TypeName', 'Cpu_brand', 'Gpu_brand', 'Os'])

df_encoded.head()


,Ram,Weight,Price,TouchScreen,Ips,Ppi,HDD,SSD,Company_Acer,Company_Apple,...,Cpu_brand_Intel Core i3,Cpu_brand_Intel Core i5,Cpu_brand_Intel Core i7,Cpu_brand_Other Intel Processor,Gpu_brand_AMD,Gpu_brand_Intel,Gpu_brand_Nvidia,Os_Mac,Os_Others,Os_Windows
0,8,1.37,11.175755,0,1,226.983005,0,128,False,True,...,False,True,False,False,False,True,False,True,False,False
1,8,1.34,10.776777,0,0,127.677940,0,0,False,True,...,False,True,False,False,False,True,False,True,False,False
2,8,1.86,10.329931,0,0,141.211998,0,256,False,False,...,False,True,False,False,False,True,False,False,True,False
3,16,1.83,11.814476,0,1,220.534624,0,512,False,True,...,False,False,True,False,True,False,False,True,False,False
4,8,1.37,11.473101,0,1,226.983005,0,256,False,True,...,False,True,False,False,False,True,False,True,False,False


**NO 2 ( A )**

In [ ]:
correlation_matrix = df_encoded.corr()

correlation_price = correlation_matrix['Price'].sort_values(ascending=False)

print(correlation_price)


Price                              1.000000
Ram                                0.681019
SSD                                0.658962
Cpu_brand_Intel Core i7            0.593680
Ppi                                0.480621
TypeName_Gaming                    0.362714
Gpu_brand_Nvidia                   0.357440
TypeName_Ultrabook                 0.306570
Os_Windows                         0.294543
Ips                                0.279483
TypeName_Workstation               0.206804
TouchScreen                        0.194436
Company_MSI                        0.188730
Weight                             0.151861
Company_Razer                      0.133579
TypeName_2 in 1 Convertible        0.093821
Os_Mac                             0.093400
Company_Apple                      0.093400
Company_Dell                       0.084786
Company_Toshiba                    0.068380
Company_LG                         0.062291
Company_Microsoft                  0.053002
Company_Samsung                 

**NO 2 ( B )**

In [ ]:
X = df_encoded.drop('Price', axis=1)
y = df_encoded['Price']


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print(coefficients)


Mean Squared Error (MSE): 0.08184524529339027
                                 Coefficient
Ram                                 0.029842
Weight                              0.078084
TouchScreen                        -0.092329
Ips                                 0.047933
Ppi                                 0.002550
HDD                                -0.000020
SSD                                 0.000500
Company_Acer                       -0.112619
Company_Apple                       0.122096
Company_Asus                        0.005759
Company_Chuwi                      -0.494322
Company_Dell                        0.051070
Company_Fujitsu                    -0.073359
Company_Google                      0.331813
Company_HP                          0.104576
Company_Huawei                     -0.100511
Company_LG                          0.350489
Company_Lenovo                      0.010714
Company_MSI                         0.173259
Company_Mediacom                   -0.439319
Company_M

**NO 2 ( C )**

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('laptop_data_cleaned.csv')

label_encoder = LabelEncoder()
df['TouchScreen'] = label_encoder.fit_transform(df['TouchScreen'])
df['Ips'] = label_encoder.fit_transform(df['Ips'])

df_encoded = pd.get_dummies(df, columns=['Company', 'TypeName', 'Cpu_brand', 'Gpu_brand', 'Os'])

df_encoded.head()


,Ram,Weight,Price,TouchScreen,Ips,Ppi,HDD,SSD,Company_Acer,Company_Apple,...,Cpu_brand_Intel Core i3,Cpu_brand_Intel Core i5,Cpu_brand_Intel Core i7,Cpu_brand_Other Intel Processor,Gpu_brand_AMD,Gpu_brand_Intel,Gpu_brand_Nvidia,Os_Mac,Os_Others,Os_Windows
0,8,1.37,11.175755,0,1,226.983005,0,128,False,True,...,False,True,False,False,False,True,False,True,False,False
1,8,1.34,10.776777,0,0,127.677940,0,0,False,True,...,False,True,False,False,False,True,False,True,False,False
2,8,1.86,10.329931,0,0,141.211998,0,256,False,False,...,False,True,False,False,False,True,False,False,True,False
3,16,1.83,11.814476,0,1,220.534624,0,512,False,True,...,False,False,True,False,True,False,False,True,False,False
4,8,1.37,11.473101,0,1,226.983005,0,256,False,True,...,False,True,False,False,False,True,False,True,False,False


In [ ]:
X = df_encoded.drop('Price', axis=1)
y = df_encoded['Price']


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")


Mean Squared Error (MSE): 0.08184524529339027


In [ ]:
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print(coefficients)


                                 Coefficient
Ram                                 0.029842
Weight                              0.078084
TouchScreen                        -0.092329
Ips                                 0.047933
Ppi                                 0.002550
HDD                                -0.000020
SSD                                 0.000500
Company_Acer                       -0.112619
Company_Apple                       0.122096
Company_Asus                        0.005759
Company_Chuwi                      -0.494322
Company_Dell                        0.051070
Company_Fujitsu                    -0.073359
Company_Google                      0.331813
Company_HP                          0.104576
Company_Huawei                     -0.100511
Company_LG                          0.350489
Company_Lenovo                      0.010714
Company_MSI                         0.173259
Company_Mediacom                   -0.439319
Company_Microsoft                   0.318157
Company_Ra